In [10]:
dayObs = 'Today'
run = "None"
sensor='R10_S10'

In [11]:
from lsst.daf.butler import Butler
from astropy.table import Table
from astropy.time import Time, TimeDelta
import tabulate
import numpy

In [12]:
if dayObs=='Today':
    dayObs = (Time.now() - TimeDelta(0.5, format='jd')).iso[:10].replace('-','')

In [13]:
butler = Butler('embargo_new', collections=[
    'LSSTCam/raw/all',
    'LSSTCam/calib/unbounded',
    'LSSTCam/photodiode'
])

In [14]:
detector=list(butler.registry.queryDimensionRecords("detector",instrument='LSSTCam',where=f"detector.full_name='{sensor}'"))[0].id

In [15]:
exposureList = []

if run == "None":
    where=f"exposure.day_obs={dayObs} and instrument='LSSTCam'"
else:
    where=f"exposure.science_program='{run}' and instrument='LSSTCam'"    

for record in butler.registry.queryDimensionRecords("exposure", where=where):
    nimage=len(list(butler.registry.queryDatasets('raw',dataId=record.dataId)))
    hasPD=len(list(butler.registry.queryDatasets('photodiode',dataId=record.dataId)))
    exposureList.append([record.id, record, nimage, hasPD])
exposureList.sort(key=lambda x: x[0])
table = Table(rows=[ (
            f'<a href=http://lsstcam-mcm.cp.lsst.org/FITSInfo/view.html?image={record.obs_id} target="_blank">{record.obs_id}</a>',
            record.timespan.begin.iso,
            record.seq_num,
            record.observation_type,
            record.science_program,
            record.observation_reason,
            record.exposure_time,
            record.physical_filter,
            nimage,
            f'<a href=https://usdf-rsp-dev.slac.stanford.edu/times-square/github/lsst/CameraTimesSquare/Run7/CheckPDTraces?exposure={record.id} target="_blank" >PD</a>' if hasPD else "",
            f'<a href=https://usdf-rsp-dev.slac.stanford.edu/times-square/github/lsst/CameraTimesSquare/Run7/FitsHeader?expid={record.obs_id}&detector={detector} target="_blank" >meta</a>',    
                     )
            for [id,record,nimage, hasPD] in exposureList ]
            )

In [16]:
subtable=Table(rows=[  ( record.obs_id, record.seq_num, nimage ) for [id,record,nimage, hasPD] in exposureList ], names=(  'id', 'seqnum', 'nimage'  ) )

# Possible data transfer issue

In [17]:
for a, b in zip(subtable[:-1][numpy.diff(subtable['seqnum'])!=1], subtable[1:][numpy.diff(subtable['seqnum'])!=1]):
    print(a['id'], b['id'])

# Changes in number of images ingested

In [18]:
for a, b in zip(subtable[:-1][numpy.diff(subtable['nimage'])!=0], subtable[1:][numpy.diff(subtable['nimage'])!=0]):
    print(a['id'], a['nimage'], b['id'], b['nimage'])

MC_C_20241014_000110 197 MC_C_20241014_000111 196
MC_C_20241014_000111 196 MC_C_20241014_000112 197
MC_C_20241014_000114 197 MC_C_20241014_000115 196
MC_C_20241014_000115 196 MC_C_20241014_000116 195
MC_C_20241014_000116 195 MC_C_20241014_000117 197
MC_C_20241014_000118 197 MC_C_20241014_000119 196
MC_C_20241014_000119 196 MC_C_20241014_000120 195
MC_C_20241014_000120 195 MC_C_20241014_000121 197
MC_C_20241014_000121 197 MC_C_20241014_000122 196
MC_C_20241014_000122 196 MC_C_20241014_000123 197
MC_C_20241014_000421 197 MC_C_20241014_000422 188
MC_C_20241014_000422 188 MC_C_20241014_000423 182
MC_C_20241014_000423 182 MC_C_20241014_000424 197
MC_C_20241014_000424 197 MC_C_20241014_000425 188
MC_C_20241014_000425 188 MC_C_20241014_000426 197
MC_C_20241014_000426 197 MC_C_20241014_000427 194
MC_C_20241014_000427 194 MC_C_20241014_000428 185
MC_C_20241014_000428 185 MC_C_20241014_000429 191
MC_C_20241014_000430 191 MC_C_20241014_000431 179
MC_C_20241014_000431 179 MC_C_20241014_000432 194


# Full records

In [19]:
print(f"The number of exposures is {len(table)}")
tabulate.tabulate(table, tablefmt='unsafehtml',headers=[
                   'obs_id', 'time', 'seqnum', 'observation_type',
                   'science_program', 'observation_reason','exposure_time',
                   'physical_filter', 'nimage', 'hasPD', 'metadata'
                  ])

The number of exposures is 463


obs_id,time,seqnum,observation_type,science_program,observation_reason,exposure_time,physical_filter,nimage,hasPD,metadata
MC_C_20241014_000001,2024-10-14 12:00:59.044017,1,flat,E1367,sflat_hi,15,HIGH,197,,meta
MC_C_20241014_000002,2024-10-14 12:02:20.078017,2,flat,E1367,sflat_hi,15,HIGH,197,,meta
MC_C_20241014_000003,2024-10-14 12:03:40.980012,3,flat,E1367,sflat_hi,15,HIGH,197,,meta
MC_C_20241014_000004,2024-10-14 12:05:01.898986,4,flat,E1367,sflat_hi,15,HIGH,197,,meta
MC_C_20241014_000005,2024-10-14 12:06:22.785009,5,flat,E1367,sflat_hi,15,HIGH,197,,meta
MC_C_20241014_000006,2024-10-14 12:07:43.687004,6,flat,E1367,sflat_hi,15,HIGH,197,,meta
MC_C_20241014_000007,2024-10-14 12:09:04.590005,7,flat,E1367,sflat_hi,15,HIGH,197,,meta
MC_C_20241014_000008,2024-10-14 12:10:25.490995,8,flat,E1367,sflat_hi,15,HIGH,197,,meta
MC_C_20241014_000009,2024-10-14 12:11:46.699003,9,flat,E1367,sflat_hi,15,HIGH,197,,meta
MC_C_20241014_000010,2024-10-14 12:13:07.599993,10,flat,E1367,sflat_hi,15,HIGH,197,,meta
